In [1]:
!pip install git+https://github.com/matterhorn-studio/MHSapi.git

  Cloning https://github.com/matterhorn-studio/MHSapi.git to /tmp/pip-req-build-f9q2wb90
  Running command git clone --filter=blob:none --quiet https://github.com/matterhorn-studio/MHSapi.git /tmp/pip-req-build-f9q2wb90
  Resolved https://github.com/matterhorn-studio/MHSapi.git to commit 1a683858079426cd7de40a7a3ab9474df278bd24
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.4/305.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

In [2]:
import pydantic
pydantic.__version__

'2.4.2'

In [3]:
# Parameters, to be overwritten by papermill when run as nextflow pipeline
project_id = "1379"
opt_run_id = "509"
token = "3ebdef55ef6fb74ee4eeb1fb30a243f6b23b2deab3f1983f21b5231086b3efa0"
base_url = "https://matterhorn.studio/"

In [4]:
# Packages
import sys
print(sys.path)
import json

import numpy as np
import pandas as pd

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [5]:
# 1. Initialise API client
import MHSapi
from MHSapi.MHSapi import MHSapiClient
from importlib.metadata import version

client = MHSapi.MHSapi.MHSapiClient(token=token, base_url=base_url)
print(version('MHSapi'))
print(MHSapi.__file__)
print(dir(MHSapi))
object_methods = [method_name for method_name in dir(client)
                  if callable(getattr(client, method_name))]
print(object_methods)

projects = client.experiments_list()
project = [p for p in projects if int(p.id) == int(project_id)][0]
parameters = client.parameters_list(project)


BASE URL: https://matterhorn.studio/
0.0.2
/usr/local/lib/python3.10/dist-packages/MHSapi/__init__.py
['MHSapi', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']
['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'experiment_data', 'experiment_update_data', 'experiments_create', 'experiments_list', 'get_base_url', 'open_experiment', 'opt_run_list', 'parameters_create', 'parameters_list']


In [6]:
# 2. Download dataset
dataset = client.experiment_data(project)
print("Data")
print(dataset)

Data
      import_timestamp  disabled  Parameter 1 Parameter 2 Parameter 3  \
0         1.707596e+09     False            0         0.0         1.0   
1         1.707596e+09     False            0         0.0         1.0   
2         1.707596e+09     False            0         0.0         1.0   
3         1.707596e+09     False            0         0.0         1.0   
4         1.707596e+09     False            0         0.0         1.0   
...                ...       ...          ...         ...         ...   
1194      1.707596e+09     False            0         0.0      1.1058   
1195      1.707596e+09     False            0         0.0      1.0168   
1196      1.707596e+09     False            0         0.0       1.103   
1197      1.707596e+09     False            0         0.0       1.122   
1198      1.707596e+09     False            0         0.0      0.9902   

      Parameter 4  Parameter 5  Parameter 6  Parameter 7  Parameter 8  ...  \
0             0.0       0.0000          

In [7]:
# 3. Get OptApp options

opt_runs = client.opt_run_list(project)
print(opt_runs)
opt_run = [p for p in opt_runs if int(p.id) == int(opt_run_id)][0]
print(opt_run)
#type = type(opt_run.run_options)
#method_list = [func for func in dir(type) if callable(getattr(type, func))]
#print(method_list)
run_options = json.loads(opt_run.run_options_string.replace("'","\""))

batch_size = 1
if 'batch_size' in run_options.keys():
    batch_size = run_options['batch_size']
batch_size

[OptimisationRun(id=509, experiment=1379, opt_app=24, run_options_string="{'sum_constraints': [{'sum_value': '1', 'parameters': ['Parameter 1', 'Parameter 2']}], 'group_activations': [{'parameters': ['Parameter 2'], 'maximum_active': '2', 'minimum_active': '1'}]}"), OptimisationRun(id=510, experiment=1379, opt_app=22, run_options_string="{'batch_size': 3}")]
id=509 experiment=1379 opt_app=24 run_options_string="{'sum_constraints': [{'sum_value': '1', 'parameters': ['Parameter 1', 'Parameter 2']}], 'group_activations': [{'parameters': ['Parameter 2'], 'maximum_active': '2', 'minimum_active': '1'}]}"


1

In [ ]:
|




# 4. Do BO
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize

inputs = [p for p in parameters if p.outcome == False and p.timestamp == False]
outcome = [p for p in parameters if p.outcome == True][0]
X = dataset[[i.parameter_text for i in inputs]]
Y = dataset[[outcome.parameter_text]]
train_X = torch.tensor(X.to_numpy(dtype=np.float64))
train_Y = torch.tensor(Y.to_numpy(dtype=np.float64))
#train_Y = standardize(train_Y)
print(train_X.shape)
print(train_X)
print(train_Y.shape)
print(train_Y)

gp = SingleTaskGP(train_X, train_Y, input_transform=Normalize(d=train_X.shape[-1]), outcome_transform=Standardize(m=train_Y.shape[-1]))

mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

#from botorch.acquisition import UpperConfidenceBound
#UCB = UpperConfidenceBound(gp, beta=0.1)

from botorch.optim import optimize_acqf

from botorch import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import (
    qExpectedImprovement,
    qNoisyExpectedImprovement,
)
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning

upper_bounds = torch.tensor([p.upper_bound for p in inputs])
lower_bounds = torch.tensor([p.lower_bound for p in inputs])
bounds = torch.stack([lower_bounds, upper_bounds])
print(f"batch_size={batch_size}")

SMOKE_TEST = False
MC_SAMPLES = 256 if not SMOKE_TEST else 32
qmc_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

qNEI = qNoisyExpectedImprovement(
    model=gp,
    X_baseline=train_X,
    sampler=qmc_sampler,
)
candidates, acq_value = optimize_acqf(
    acq_function=qNEI, bounds=bounds, q=int(batch_size), num_restarts=5, raw_samples=20,
)
candidates  # tensor([0.4887, 0.5063])
print("Candidates (raw):")
print(candidates.detach())

candidates = pd.DataFrame(candidates.numpy())
candidates.columns = [input.parameter_text for input in inputs]
candidates[outcome.parameter_text] = np.nan
candidates["opt_run_id"] = opt_run_id

print(candidates)


In [ ]:
# 5. Upload candidates
client.experiment_update_data(project, candidates)
